Requires `TX-input-data.csv` available on Box at `data/model inputs and outputs`

In [1]:
import pandas as pd
import numpy as np

In [2]:
usgs = pd.read_csv("TX-input-data.csv")

In [3]:
usgs.head()

,year,n_turbine,t_cap_mw,ylat,xlong,avg_concrete_per_mw
0,1999,123,0.723740,-100.376937,31.988169,825.004422
1,2000,0,0.723740,-100.376937,31.988169,825.004422
2,2001,851,1.051492,-100.376937,31.988169,624.068080
3,2002,5,0.660000,-100.376937,31.988169,624.068080
4,2003,186,1.077957,-100.376937,31.988169,545.485129


In [4]:
def blade_mass_tonnes_map(row):
    t_cap_mw=row['t_cap_mw']
    
    if t_cap_mw < 1.0:
        tonnes = t_cap_mw * 8.43
        
    elif t_cap_mw >= 1.0 and t_cap_mw < 1.5:
        tonnes = t_cap_mw * 12.37
        
    elif t_cap_mw >= 1.5 and t_cap_mw < 2.0:
        tonnes = t_cap_mw * 13.34
        
    elif t_cap_mw >= 2.0 and t_cap_mw < 5.0:
        tonnes = t_cap_mw * 13.41
        
    else:  # > 5.0 MW
        tonnes = t_cap_mw * 12.58
        
    return tonnes

In [5]:
blade_mass_tonnes = usgs.apply(blade_mass_tonnes_map, axis=1)
usgs["blade_mass_tonnes"] = blade_mass_tonnes

In [6]:
usgs.head()

,year,n_turbine,t_cap_mw,ylat,xlong,avg_concrete_per_mw,blade_mass_tonnes
0,1999,123,0.723740,-100.376937,31.988169,825.004422,6.101127
1,2000,0,0.723740,-100.376937,31.988169,825.004422,6.101127
2,2001,851,1.051492,-100.376937,31.988169,624.068080,13.006961
3,2002,5,0.660000,-100.376937,31.988169,624.068080,5.563800
4,2003,186,1.077957,-100.376937,31.988169,545.485129,13.334328


In [7]:
def foundation_mass_tonnes(row):
    # calculates the *cement only* mass going into new foundations
    # aggregate, water, steel rebar are not included here
    year=row['year']
    t_cap_mw=row['t_cap_mw']
    
    fraction_cement_in_concrete=0.125
    
    if year < 2020:
        # use historical data already in the data 
        tonnes_cement = fraction_cement_in_concrete * row['avg_concrete_per_mw'] * t_cap_mw
    else:
        # calculate from projection based on historical data
        tonnes_cement = fraction_cement_in_concrete * 611.29 * t_cap_mw**-0.735
    
    return tonnes_cement    

In [8]:
foundation_cement_tonnes = usgs.apply(foundation_mass_tonnes, axis=1)
usgs['foundation_mass_tonnes'] = foundation_cement_tonnes

In [9]:
usgs.head()

,year,n_turbine,t_cap_mw,ylat,xlong,avg_concrete_per_mw,blade_mass_tonnes,foundation_mass_tonnes
0,1999,123,0.723740,-100.376937,31.988169,825.004422,6.101127,74.636071
1,2000,0,0.723740,-100.376937,31.988169,825.004422,6.101127,74.636071
2,2001,851,1.051492,-100.376937,31.988169,624.068080,13.006961,82.025352
3,2002,5,0.660000,-100.376937,31.988169,624.068080,5.563800,51.485617
4,2003,186,1.077957,-100.376937,31.988169,545.485129,13.334328,73.501188


In [10]:
# remove column that's no longer needed
usgs.drop(columns='avg_concrete_per_mw',inplace=True)

In [11]:
usgs.head()

,year,n_turbine,t_cap_mw,ylat,xlong,blade_mass_tonnes,foundation_mass_tonnes
0,1999,123,0.723740,-100.376937,31.988169,6.101127,74.636071
1,2000,0,0.723740,-100.376937,31.988169,6.101127,74.636071
2,2001,851,1.051492,-100.376937,31.988169,13.006961,82.025352
3,2002,5,0.660000,-100.376937,31.988169,5.563800,51.485617
4,2003,186,1.077957,-100.376937,31.988169,13.334328,73.501188


In [12]:
usgs.loc[:100, :]

,year,n_turbine,t_cap_mw,ylat,xlong,blade_mass_tonnes,foundation_mass_tonnes
0,1999,123,0.723740,-100.376937,31.988169,6.101127,74.636071
1,2000,0,0.723740,-100.376937,31.988169,6.101127,74.636071
2,2001,851,1.051492,-100.376937,31.988169,13.006961,82.025352
3,2002,5,0.660000,-100.376937,31.988169,5.563800,51.485617
4,2003,186,1.077957,-100.376937,31.988169,13.334328,73.501188
5,2004,0,1.077957,-100.376937,31.988169,13.334328,73.501188
6,2005,434,1.559171,-100.376937,31.988169,20.799335,75.119648
7,2006,397,1.865743,-100.376937,31.988169,24.889013,86.088484
8,2007,980,1.744694,-100.376937,31.988169,23.274216,89.811861
9,2008,1694,1.579870,-100.376937,31.988169,21.075468,85.388662


In [13]:
usgs.to_csv("TX_input_data_with_masses.csv", index=False)